In [40]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import random
import io
import imageio
import glob
import imutils
import cv2
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
from tqdm import tqdm
import math

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

%matplotlib inline

print("Done")

Done


In [60]:
def extract_sub(new_image):
    t_lower = 110
    t_upper = 254
    alpha = 1.001
    beta = -0.99
    new_image = np.array(new_image)
    
    h, w = np.shape(new_image)[:2]
    
    new_image = cv2.convertScaleAbs(new_image, alpha=alpha, beta=beta)

    # sharpen
    sharp_image = cv2.GaussianBlur(new_image, [0, 0], 3)
    new_image = cv2.addWeighted(new_image, 2, sharp_image, -1, 0)
    #new_image = ((cv2.addWeighted(new_image, sharp_value, sharp_image, 1 - sharp_value, 0)) / 255).astype('uint8')

    aperture_size = 3 # Aperture size (default = 3)
    L2Gradient = False # Boolean (default = False)
    edges = cv2.Canny(np.array(new_image), params[0], params[1], apertureSize = aperture_size, L2gradient = L2Gradient) # 150, 75 confirmed

    ymin = h
    xmin = w
    ymax = 0
    xmax = 0
    coords = [ymin, xmin, ymax, xmax]
    
    # Find left bound/xmin
    left_bound = np.zeros([h, w]) # 150, 75 confirmed
    row_track = 0
    while row_track <= h - 1:
        column_track = 0
        while column_track <= w - 1:
            value = edges[row_track, column_track]
            if (value > 0):
                left_bound[row_track][column_track:(w - 1)] = 1
                if (column_track < coords[1]):
                    coords[1] = column_track
                column_track = w
            column_track = column_track + 1
        row_track = row_track + 1
    
    # Find right bound/xmax
    right_bound = np.zeros([h, w])
    row_track = 0
    while row_track <= h - 1:
        column_track = w - 1
        while column_track >= 0:
            value = edges[row_track, column_track]
            if (value > 0):
                right_bound[row_track][0:column_track] = 1
                if (column_track > coords[3]):
                    coords[3] = column_track
                column_track = 0
            column_track = column_track - 1
        row_track = row_track + 1

    # Find top bound/ymin
    top_bound = np.zeros([w, h]) # 75, 150 confirmed
    column_track = 0
    while column_track <= w - 1:
        row_track = 0
        while row_track <= h - 1:
            value = edges[row_track, column_track]
            if (value > 0):
                top_bound[column_track][row_track:(h - 1)] = 1
                if (row_track < coords[0]):
                    coords[0] = row_track
                row_track = h
            row_track = row_track + 1
        column_track = column_track + 1
    top_bound = np.transpose(top_bound)
    
    # Find bottom bound/ymax
    bottom_bound = np.zeros([w, h])
    column_track = 0
    while column_track <= w - 1:
        row_track = h - 1
        while row_track >= 0:
            value = edges[row_track, column_track]
            if (value > 0):
                bottom_bound[column_track][0:row_track] = 1
                if (row_track > coords[2]):
                    coords[2] = row_track
                row_track = 0
            row_track = row_track - 1
        column_track = column_track + 1
    bottom_bound = np.transpose(bottom_bound)
    
    mask = (top_bound * bottom_bound * right_bound * left_bound).astype('uint8')

    # smooth out the mask
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations = 1)
    mask = cv2.medianBlur(mask, 5)
    mask = np.array([[math.ceil(abs(j) / 255) for j in i] for i in mask]).astype('uint8')
    print(np.shape(mask))
    mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2).astype(np.uint8)
    
    #new_image = new_image * mask
    print(np.array(coords))
    #new_image = Image.fromarray(new_image.astype(np.uint8))
    #new_image = new_image.crop((coords[1], coords[0], coords[3], coords[2]))

    return mask

print("Done")

Done


In [37]:
def cost(ideal_mask, test_image, params):
    return sum([sum(k) for k in [[math.ceil(abs(j) / 255) for j in i] for i in (ideal_mask - extract_sub(test_image, params))]])

def grad_desc(test_image, ideal_mask, params):
    iterations = 250
    delta = 1e-3
    eps = 0.001
    gamma = 7e-6

    # m_norm is a function f(margin, sharp, a, b)
    # Params - an [N, 1] array of parameters to be optimized
    # Gamma - weighting factor (decrease when diverging)
    # Delta - step size
    # eps - early cutoff convergence threshold

    new_params = np.ones(len(params))
    dfdm_norm = np.ones(len(params))
    m_norm = cost(ideal_mask, test_image, params)

    i = 0
    best = params
    m_norm_min = m_norm
    print('Starting m_norm is: ' + str(m_norm_min), flush=True)

    with tqdm(total=iterations, desc="Gradient Descent", unit="iter") as pbar:
        #while abs(sum(dfdm_norm)) > eps and i < iterations:
        while i < iterations:
            m_norm = cost(ideal_mask, test_image, params)
            if m_norm < m_norm_min:
                m_norm_min = m_norm
                best = params
            for j in range(len(params)): #calculating new gradient parameter by parameter
                new_params = params
                new_params[j] = params[j] + delta
                dfdm_norm[j] = (cost(ideal_mask, test_image, new_params) -  m_norm) / delta
            params = params - gamma * dfdm_norm
            pbar.update(1)
            pbar.set_postfix(best_m_norm=m_norm_min, current_m_norm=m_norm)#, mar=opt_margin, shp=opt_sharp, a=opt_alpha, b=opt_beta)
            i = i + 1
    print('Final t_upper is: ' + str(params[0]), flush=True)
    print('Final t_lower value is: ' + str(params[1]), flush=True)
    print('Final alpha is: ' + str(params[2]), flush=True)
    print('Final beta is: ' + str(params[3]), flush=True)
    print('Final m_norm is: ' + str(m_norm_min), flush=True)
    
    return params, cost(ideal_mask, test_image, params)

print("Done")

Done


In [61]:
test_image = Image.open("C:/Users/khard/Documents/GitHub/jupyter/PollenCounter/Archive/BeeDataset_20201121/images_150/0fRg4nOAzBV4GVCk26UaIVHUhQ8zwcwd.jpeg")
# assert test_image is not None, "file could not be read, check with os.path.exists()"
# ideal_mask = ((cv2.cvtColor(np.array(Image.open("C:/Users/khard/Documents/GitHub/jupyter/PollenCounter/BeeMask.png")), cv2.COLOR_BGR2GRAY))/255).astype('uint8')
# h, w, _ = np.shape(test_image)

# t_lower = 110 # Lower Threshold
# t_upper = 254 # Upper threshold (higher yields less detail)
# # sharp = 1.596
# # a = 0.136
# # b = -111
# a = 1
# b = -0.1
# params = [t_upper, t_lower, a, b]
# #best = params
# #best, m_norm = grad_desc(test_image, ideal_mask, params)

new_image = extract_sub(test_image)#, best)
#m_norm = cost(ideal_mask, test_image, params)
#new_mask3 = np.repeat(new_mask[:, :, np.newaxis], 3, axis=2).astype(np.uint8)
#new_image = new_mask3 * test_image

# plt.imshow(new_image)
# plt.show

(150, 75)
[  0   0 126  74]


In [72]:
print(params)

[254.0004, 110.0004, 1.0004, -0.09960000000000001]
